In [ ]:
from google.colab import drive
! pip install pytorch-lightning --quiet
drive.mount('/content/drive/')
%cd ./drive/MyDrive/Colab\ Notebooks/mlearning

In [ ]:
from util import *
from sklearn.svm import SVC, LinearSVC
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
import seaborn as sns
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

In [ ]:
ds = SeedDataset()
print(f'training_set size: {len(ds)}')
test_ds = SeedDataset(True)
print(f'testing_set size: {len(test_ds)}')


In [ ]:
clf =  SGDClassifier(max_iter=1000, tol=1e-3)

batch_size = 100 #73257 = all
sdm = SVHNDataModule(batch_size=batch_size)
sdm.setup()

In [ ]:

subset_size = 10000
n_batch = subset_size / batch_size
print(n_batch)
x_train = list()
y_train = list()
i = 0
for batch in sdm.train_dataloader():
    x,y = batch
    # input()
    x = x.view(x.size(0), -1)
    y = np.squeeze(y,1)
    x_train.append(x)
    y_train.append(y)
len(x_train)
x_train = np.concatenate(x_train,0)
y_train = np.concatenate(y_train,0)
x_train, _, y_train, __ = train_test_split(x_train, y_train, train_size=5000, shuffle=True)
x_train[0].shape

In [ ]:
x_test.shape[1]

In [ ]:
x_test = list()
y_test = list()
test_size = 10000
# if(i<test_size):
for batch in sdm.test_dataloader():
    x,y = batch
    x = x.view(x.size(0), -1)
    y = np.squeeze(y,1)
    x_test.append(x)
    y_test.append(y)
x_test = np.concatenate(x_test,0)
y_test = np.concatenate(y_test,0)
x_test, _, y_test, __ = train_test_split(x_test, y_test, train_size=2000, shuffle=True)
x_test[0].shape

In [ ]:
pca = PCA(n_components=0.9)
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)

scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
# pca = PCA(n_components=2)
# # pca_res = pca.fit_transform(x_test)
plt.scatter(x_test[:, 0], x_test[:, 1], c=y_test,cmap=plt.cm.get_cmap('Set1', 10))

In [ ]:
# plt.figure(figsize=(20,4))
# for index, (image, label) in enumerate(zip(x_train[0:5], y_train[0:5])):
#  plt.subplot(1, 5, index + 1)
#  plt.imshow(np.reshape(image, (32,32,3)), cmap=plt.cm.gray)
#  plt.title('Training: %i\n' % label, fontsize = 20)

# SVM

In [ ]:
target_names = ['class_1', 'class_2',
                'class_3', 'class_4', 'class_5',
                'class_6', 'class_7', 'class_8','class_9','class_0']


def train_model(m):
    m.fit(x_train,y_train)
    print(m.score(x_test,y_test))

# svm_linear = SVC(kernel='linear',cache_size = 500)
# train_model(svm_linear)
# Grid search is used to find the gamma and c hyperparameter
# c = 1, 0.
# c = 10, 0.1816

In [ ]:
# import pickle
# filename = 'finalized_model.sav'
# # pickle.dump(svm_linear, open(filename, 'wb'))
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(x_test, y_test)
# print(result)

In [ ]:
# weights = loaded_model.coef_
def imsho_features(weights):
    plt.figure(figsize=(10,10))
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plo_feature(weights,i)
    plt.show()
def plo_feature(w,number=0):
    no = number-1
    image = w[no]
    min = np.abs(image).max()
    img = (image).reshape(32,32)
    plt.imshow(img, vmin=-min,vmax=min, cmap="viridis",interpolation='nearest')
# imsho_features(weights)


In [ ]:

# res = svm_linear.predict(x_test)
# print(classification_report(y_test, res, target_names=target_names))

In [ ]:
import pickle
filename = 'finalized_rbf_model.sav'
svm_rbf = SVC(kernel='rbf',cache_size = 500)
train_model(svm_rbf)
pickle.dump(svm_rbf, open(filename, 'wb'))

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_regions(X,y,classifier,test_idx=None,resolution=0.02):
    
    # Initialise the marker types and colors
    markers = ('s','x','o','^','v')
    colors = ('red','blue','lightgreen','gray','cyan')
    color_Map = ListedColormap(colors[:len(np.unique(y))]) #we take the color mapping correspoding to the 
                                                            #amount of classes in the target data
    
    # Parameters for the graph and decision surface
    x1_min = X[:,0].min() - 1
    x1_max = X[:,0].max() + 1
    x2_min = X[:,1].min() - 1
    x2_max = X[:,1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min,x1_max,resolution),
                           np.arange(x2_min,x2_max,resolution))
    
    Z = classifier.predict(np.array([xx1.ravel(),xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    
    plt.contour(xx1,xx2,Z,alpha=0.4,cmap = color_Map)
    plt.xlim(xx1.min(),xx1.max())
    plt.ylim(xx2.min(),xx2.max())
    
    # Plot samples
    X_test, Y_test = X[test_idx,:], y[test_idx]
    
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x = X[y == cl, 0], y = X[y == cl, 1],
                    alpha = 0.8, c = color_Map(idx),
                    marker = markers[idx], label = cl
                   )
x_train.shape[1]
plot_decision_regions(X = x_test,
                       y = y_test
                      , classifier = svm_rbf
                      )

In [ ]:
# svm_poly = SVC(kernel='poly',degree = 3,cache_size = 500)
# train_model(svm_poly)

In [ ]:
svm_poly = SVC(kernel='poly',degree = 4,cache_size = 500)
# train_model(svm_poly)

In [ ]:
svm_poly = SVC(kernel='poly',degree = 5,cache_size = 1000)
# train_model(svm_poly)

In [ ]:


fig, ax = plt.subplots()
# title for the plots
title = ('Decision surface of linear SVC ')
# Set-up grid for plotting.
X0, X1 = pca_res[:, 0], pca_res[:, 1]
xx, yy = make_meshgrid(X0, X1)

plot_contours(ax, svm_poly, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
ax.set_ylabel('y label here')
ax.set_xlabel('x label here')
ax.set_xticks(())
ax.set_yticks(())
ax.set_title(title)
ax.legend()
plt.show()

# Tensorflow board